In [1]:
import pandas as pd

In [70]:
train = pd.read_csv("./kaggle4th/train.csv") 
test = pd.read_csv("./kaggle4th/test.csv") 
sub = pd.read_csv("./kaggle4th/sample_submission.csv") 

train.shape, test.shape, sub.shape

((26049, 16), (6512, 15), (6512, 2))

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26049 entries, 0 to 26048
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26049 non-null  int64 
 1   age             26049 non-null  int64 
 2   workclass       26049 non-null  object
 3   fnlwgt          26049 non-null  int64 
 4   education       26049 non-null  object
 5   education_num   26049 non-null  int64 
 6   marital_status  26049 non-null  object
 7   occupation      26049 non-null  object
 8   relationship    26049 non-null  object
 9   race            26049 non-null  object
 10  sex             26049 non-null  object
 11  capital_gain    26049 non-null  int64 
 12  capital_loss    26049 non-null  int64 
 13  hours_per_week  26049 non-null  int64 
 14  native_country  26049 non-null  object
 15  income          26049 non-null  object
dtypes: int64(7), object(9)
memory usage: 3.2+ MB


In [5]:
train.loc[ train['income']=='>50K', 'target'] = 1
train.loc[ train['income']=='<=50K', 'target'] = 0
train['target'] = train['target'].astype("int")
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26049 entries, 0 to 26048
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              26049 non-null  int64 
 1   age             26049 non-null  int64 
 2   workclass       26049 non-null  object
 3   fnlwgt          26049 non-null  int64 
 4   education       26049 non-null  object
 5   education_num   26049 non-null  int64 
 6   marital_status  26049 non-null  object
 7   occupation      26049 non-null  object
 8   relationship    26049 non-null  object
 9   race            26049 non-null  object
 10  sex             26049 non-null  object
 11  capital_gain    26049 non-null  int64 
 12  capital_loss    26049 non-null  int64 
 13  hours_per_week  26049 non-null  int64 
 14  native_country  26049 non-null  object
 15  income          26049 non-null  object
 16  target          26049 non-null  int32 
dtypes: int32(1), int64(7), object(9)
memory usage: 3.3

In [7]:
train.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,target
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K,1
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K,0
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K,0
3,3,21,Private,151158,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,25,United-States,<=50K,0
4,4,24,Private,122234,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,20,?,<=50K,0


In [9]:
sel = ['id', 'age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']

X = train[sel]
y = train['target']

test_X = test[sel]

In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                 stratify=train.target,
                                 random_state=42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(19536, 7) (6513, 7) (19536,) (6513,)


In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [20]:
model = LogisticRegression().fit(X_train, y_train)
score=cross_val_score(model, X_test, y_test, scoring='f1')
print("학습용 : ", model.score(X_train, y_train) )
print("테스트용 : ", model.score(X_test, y_test) )
print("f1 : ", score, score.mean())

학습용 :  0.7971437346437347
테스트용 :  0.7996315062183326
f1 :  [0.4        0.39179954 0.41704036 0.43280182 0.38901602] 0.40613154875872554


In [24]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier().fit(X_train, y_train)
pred = model.predict(X_test)
score=cross_val_score(model, X_test, y_test, scoring='f1')
print("학습용 : ", model.score(X_train, y_train) )
print("테스트용 : ", model.score(X_test, y_test) )
print("f1 : ", score, score.mean())

학습용 :  1.0
테스트용 :  0.8320282511899278
f1 :  [0.56439394 0.59642147 0.5981982  0.57854406 0.58426966] 0.5843654665978261


In [41]:
from sklearn.metrics import classification_report
print(classification_report(y_test,pred, target_names=['<=50K','>50K']))

              precision    recall  f1-score   support

       <=50K       0.85      0.96      0.90      4937
        >50K       0.80      0.47      0.59      1576

    accuracy                           0.84      6513
   macro avg       0.83      0.71      0.75      6513
weighted avg       0.84      0.84      0.83      6513



In [27]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier().fit(X_train, y_train)
pred = model.predict(X_test)
score=cross_val_score(model, X_test, y_test, scoring='f1')
print("학습용 : ", model.score(X_train, y_train) )
print("테스트용 : ", model.score(X_test, y_test) )
print("f1 : ", score, score.mean())

학습용 :  0.8183353808353808
테스트용 :  0.7755258713342545
f1 :  [0.29357798 0.30555556 0.29716981 0.28909953 0.28703704] 0.2944879823262148


In [33]:
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier().fit(X_train, y_train)
pred = model.predict(X_test)
score=cross_val_score(model, X_test, y_test, scoring='f1')
print("학습용 : ", model.score(X_train, y_train) )
print("테스트용 : ", model.score(X_test, y_test) )
print("f1 : ", score, score.mean())

학습용 :  0.8421887796887797
테스트용 :  0.8429295255642562
f1 :  [0.59770115 0.59274194 0.60074627 0.60159363 0.61111111] 0.6007788180349988


In [40]:
from sklearn.metrics import confusion_matrix
con=confusion_matrix(y_test, pred)
print(con)
print((con[0,0]+con[1,1])/con.sum())
print(con[0,0]/(con[0,0]+con[0,1]))
print(con[0,0]/(con[0,0]+con[1,0]))
print(con[1,1]/(con[0,1]+con[1,1]))
print(con[0,1]/(con[0,1]+con[1,1]))

[[4757  180]
 [ 843  733]]
0.8429295255642562
0.9635406117075147
0.8494642857142857
0.8028477546549836
0.19715224534501644


In [71]:
y = train['income']
test['income'] = 'blank'

In [72]:
print(train.shape, test.shape)
all_dat = pd.concat([train, test], axis=0)
print(all_dat.shape)

(26049, 16) (6512, 16)
(32561, 16)


In [73]:
all_dat.income.value_counts()

<=50K    19744
blank     6512
>50K      6305
Name: income, dtype: int64

In [74]:
all_dat.loc[ all_dat['income']=='>50K'  ,  'target' ] = 1
all_dat.loc[ all_dat['income']=='<=50K'  ,  'target' ] = 0
all_dat.loc[ all_dat['income']=='blank'  ,  'target' ] = 999

all_dat['target'] = all_dat['target'].astype('int')
all_dat.head()

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,target
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K,1
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K,0
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K,0
3,3,21,Private,151158,Some-college,10,Never-married,Prof-specialty,Own-child,White,Female,0,0,25,United-States,<=50K,0
4,4,24,Private,122234,Some-college,10,Never-married,Adm-clerical,Not-in-family,Black,Female,0,0,20,?,<=50K,0


In [75]:
all_dat['workclass'].value_counts()

Private             22696
Self-emp-not-inc     2541
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: workclass, dtype: int64

In [82]:
from sklearn.preprocessing import LabelEncoder
en_x = LabelEncoder()
all_dat['workclass_lbl'] = en_x.fit_transform(all_dat['workclass'])
all_dat.head(3)

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,native_country,income,target,workclass_lbl
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,60,United-States,>50K,1,4
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,Male,0,0,20,United-States,<=50K,0,4
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,16,United-States,<=50K,0,4


In [83]:
all_dat['education_lbl'] = en_x.fit_transform(all_dat['education'])
all_dat['marital_status_lbl'] = en_x.fit_transform(all_dat['marital_status'])
all_dat['occupation_lbl'] = en_x.fit_transform(all_dat['occupation'])
all_dat['relationship_lbl'] = en_x.fit_transform(all_dat['relationship'])
all_dat['race_lbl'] = en_x.fit_transform(all_dat['race'])
all_dat['native_country_lbl'] = en_x.fit_transform(all_dat['native_country'])
all_dat.head(3)

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,...,native_country,income,target,workclass_lbl,education_lbl,marital_status_lbl,occupation_lbl,relationship_lbl,race_lbl,native_country_lbl
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,...,United-States,>50K,1,4,11,2,12,0,4,39
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,...,United-States,<=50K,0,4,6,4,7,3,4,39
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,...,United-States,<=50K,0,4,15,4,8,3,4,39


In [84]:
mf_mapping = {"Male": 1, "Female": 2}
all_dat['sex'] = all_dat['sex'].map(mf_mapping)
all_dat.head(3)

,id,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,...,native_country,income,target,workclass_lbl,education_lbl,marital_status_lbl,occupation_lbl,relationship_lbl,race_lbl,native_country_lbl
0,0,40,Private,168538,HS-grad,9,Married-civ-spouse,Sales,Husband,White,...,United-States,>50K,1,4,11,2,12,0,4,39
1,1,17,Private,101626,9th,5,Never-married,Machine-op-inspct,Own-child,White,...,United-States,<=50K,0,4,6,4,7,3,4,39
2,2,18,Private,353358,Some-college,10,Never-married,Other-service,Own-child,White,...,United-States,<=50K,0,4,15,4,8,3,4,39


In [85]:
## 필요없는 데이터 열을 확인
## 'workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'native_country'
sel = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'native_country']
all_dat_n = all_dat.drop(sel, axis=1)
all_dat_n

,id,age,fnlwgt,education_num,sex,capital_gain,capital_loss,hours_per_week,income,target,workclass_lbl,education_lbl,marital_status_lbl,occupation_lbl,relationship_lbl,race_lbl,native_country_lbl
0,0,40,168538,9,1,0,0,60,>50K,1,4,11,2,12,0,4,39
1,1,17,101626,5,1,0,0,20,<=50K,0,4,6,4,7,3,4,39
2,2,18,353358,10,1,0,0,16,<=50K,0,4,15,4,8,3,4,39
3,3,21,151158,10,2,0,0,25,<=50K,0,4,15,4,10,3,4,39
4,4,24,122234,10,2,0,0,20,<=50K,0,4,15,4,1,1,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6507,6507,35,61343,13,1,0,0,40,blank,999,4,9,2,12,0,4,39
6508,6508,41,32185,13,1,0,0,40,blank,999,5,9,2,13,0,4,39
6509,6509,39,409189,3,1,0,0,40,blank,999,4,4,2,8,0,4,26
6510,6510,35,180342,9,1,0,0,40,blank,999,4,11,2,3,0,4,39


In [86]:
train_n = all_dat_n.loc[ (all_dat_n['target']==0) | (all_dat_n['target']==1)  , : ]
test_n = all_dat_n.loc[ all_dat_n['target']==999  ,  : ]

In [87]:
# 각각에 필요없는 컬럼을 삭제
# train(income), test(income, target)

sel_cat = ['income']

train_n = train_n.drop(['income'], axis=1)
test_n = test_n.drop(['income', 'target'], axis=1)

In [125]:
sel = ['age', 'education_num', 'sex', 'relationship_lbl', 'occupation_lbl', 'capital_gain', 'capital_loss']
X = train_n[sel]
y = train_n['target']

X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y,test_size=0.3,random_state=0)

In [126]:
model = GradientBoostingClassifier().fit(X_train,y_train)
pred = model.predict(X_test)
print(classification_report(y_test, pred, target_names=['<=50K','>50K']))

              precision    recall  f1-score   support

       <=50K       0.88      0.95      0.92      5923
        >50K       0.79      0.61      0.69      1892

    accuracy                           0.87      7815
   macro avg       0.84      0.78      0.80      7815
weighted avg       0.86      0.87      0.86      7815

